In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.attraction_constants import *

from packages.attraction.Attraction import *
from packages.file_handler_package.file_handler import *
from packages.attraction.mulProcess_attraction_scraping_proxy import *

import os
import glob
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values 

from multiprocessing import Pool
from functools import partial

In [8]:
# *** select one province from 'ALL_PROVINCE_TRIPADVISOR_DATA'
# *** so, change "Idx_of_region" everytime when scrape another province
Idx_of_region = 0
cur_region_data = ALL_PROVINCE_TRIPADVISOR_DATA[Idx_of_region]

# select first and last page to scrape
first_page = 46
last_page = 50

cur_province_en = cur_region_data[0]
cur_province_th = cur_region_data[1]
cur_province_url = cur_region_data[2]

# create list of : [1, 2, 3, ...., last_page]
page_number_list = np.arange(first_page, last_page + 1)
print("check page number list: ")
print(page_number_list)

# get list of dataframe result of all Attraction in a province
# res_allAttractions_list = []
with Pool(processes=5) as pool: 
    func_param_fix = partial(mulProcess_helper_scrape_attractions_by_province, province_url = cur_province_url, province = cur_province_th)
    res_allAttractions_list = pool.map(func_param_fix, page_number_list)
    print("end process for province --> %s " % (cur_province_th))

# concat all dataframe into a final dataframe to save in csv.
res_allAttractions_df = pd.DataFrame()
print("check len of list result dataframe --> ", len(res_allAttractions_list))
for Idx, cur_df in list(enumerate(res_allAttractions_list)):
    print("cur_df --> page : ", Idx + first_page)
    print("len cur_df --> ", len(cur_df))
    res_allAttractions_df = pd.concat([res_allAttractions_df, cur_df])

# don't forget to remove row with lat/long be zero

# remove duplicate attraction
res_allAttractions_df.drop_duplicates(subset=['name'], inplace=True)
# remove row with missing latitude
# Drop rows with 'latitude' or 'longitude' equal to 0
res_allAttractions_df = res_allAttractions_df[(res_allAttractions_df['latitude'] != 0) & (res_allAttractions_df['longitude'] != 0)]
# set new index
res_allAttractions_df.set_index(['name'], inplace=True)

# create directory to store result of scraping attraction
# for example: 'attraction_scraping\res_attraction_scraping\res_attraction_Phuket'
createDirectory(fh.STORE_ATTRACTION_SCRAPING, os.path.join('res_attraction_scraping', 'res_attraction_%s' % (cur_province_en)))

# save result dataframe to .csv
# for example: 'res_attraction_Phuket_page_1_44.csv'
res_file_name = 'res_attraction_%s_page_%s_%s.csv' % (cur_province_en, first_page, last_page)
res_path = os.path.join(fh.STORE_ATTRACTION_SCRAPING, 'res_attraction_scraping', 'res_attraction_%s' % (cur_province_en), res_file_name)
res_allAttractions_df.to_csv(res_path, encoding="utf-8")
    

check page number list: 
[46 47 48 49 50]
end process for province --> ภูเก็ต 
check len of list result dataframe -->  5
cur_df --> page :  46
len cur_df -->  5
cur_df --> page :  47
len cur_df -->  8
cur_df --> page :  48
len cur_df -->  8
cur_df --> page :  49
len cur_df -->  10
cur_df --> page :  50
len cur_df -->  7
Directory res_attraction_scraping\res_attraction_Phuket created successfully
